###### Voronoi constructor Module 

### Usefull Libraries

In [6]:
import math
import numpy as np
import sys

import matplotlib.pyplot as plt
import matplotlib.path as path
import matplotlib.colors as mcolors
import matplotlib._color_data as mcd
from matplotlib.patches import Polygon as pltpolygon
from matplotlib.collections import PatchCollection

from scipy.spatial import Voronoi, voronoi_plot_2d, Delaunay

from shapely.geometry import LineString, Polygon, MultiPolygon, Point
from shapely.ops import polygonize, unary_union

import import_ipynb
import node_linker
import solver


### Position Generation

`pos_gen(num, seed)`

Generate a pseudorandom configurations for initial and final positions

#### Parameters:
* **num&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;int**
    * Number of robots
* **seed&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;int**
    * Seed for pseudorandom
    
#### Return:
* **init&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;array of list of floats**
    * Initial coordinates for robots position
* **fin&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;array of list of floats**
    * Final coordinates for robots position
* **distances&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;list of floats**
    * Distances between initial and final positions

In [15]:
def pos_gen(num, seed):
    np.random.seed(seed)
    init = []
    fin = []
    distances = []
    radius = math.sqrt(2*num)
    offset = (1/(0.4*num) * np.random.random_sample((num*4,)) - 1/(0.4*num))
    
    for i in range(0, 4*num, 4):
        convert = (2*math.pi*i/4)/num
        init.append([round(radius*math.cos(convert) + offset[i]  ,2),
                     round(radius*math.sin(convert) + offset[i+1],2)])
        fin.append([ round(radius*math.cos(convert+math.pi) + offset[i+2],2),
                     round(radius*math.sin(convert+math.pi) + offset[i+3],2)])
        
    return {"init"  : np.array(init),
            "fin"   : np.array(fin),
            "dist"  :[np.linalg.norm(np.array(fin[i]) - np.array(init[i]))
                      for i in range(num)]
           }

### Sharpen Function

`def sharpen(head, tail)`

Calculate a new vertex when edges smaller than the safety radius appear 

#### Parameters:
* **aux1&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of floats**
    * Auxiliary coordinate for vertex1 to generate the peak 
* **vertex1&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of floats**
    * One of the edge vertices
* **vertex2&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of floats**
    * The other edge vertices
* **aux2&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of floats**
    * Auxiliary coordinate for vertex2 to generate the peak
    
#### Return:
* **peak&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of floats**
    * New coordinate that replace the edge

In [ ]:
def sharpen(aux1, vertex1, vertex2, aux2):
    # Special cases: vertical edge
    if aux1[0] == vertex1[0]:
        slope2 = (vertex2[1] - aux2[1])/(vertex2[0] - aux2[0])
        x = aux1[0]
        y = slope2*(x - aux2[0]) + aux2[1]
    elif aux2[0] == vertex2[0]:
        slope1 = (vertex1[1] - aux1[1])/(vertex1[0] - aux1[0])
        x = aux2[0]
        y = slope1*(x - aux1[0]) + aux1[1]
    # Another cases
    else:
        slope1 = (vertex1[1] - aux1[1])/(vertex1[0] - aux1[0])
        slope2 = (vertex2[1] - aux2[1])/(vertex2[0] - aux2[0])
        x = ((vertex2[1] - aux1[1] + slope1*aux1[0] - slope2*vertex2[0]) /
             (slope1 - slope2))
        y = slope1*(x - aux1[0]) + aux1[1]
    peak = np.array([x,y])
    return peak

### Collision Free Configuration Function

`def collision(positions, radius)`

Test if the initial position of robots is collision free configuration.

#### Parameters:
* **positions&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;float**
    * Positions of robots 
* **pointers&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of ints, shape `(nridges, 2)`**
    * Indices of the points between which each Voronoi ridge lies (from scipy.spatial.Voronoi.ridge_points)
* **radius&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;float**
    * radius to define the BVC from the Voronoi Diagram
    
#### Return:
* **crash&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;bool**
    * False if no collision exists

In [73]:
def collision(positions, pointers, radius):
    crash = False
    i = 0
    while i < len(pointers) and not crash:
        if np.linalg.norm(positions[pointers[i][1]]
                          - positions[pointers[i][0]]) < 2*radius:
            crash = True
        i += 1
    return crash

### Vertices Sense Function

`def getSense(vertices, previous_position)`

Get the sense of a vertices set

#### Parameters:
* **vetices&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;list of ndarrays of float**
    * Current positions of the robots.
* **previous_position&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of floats**
    * Coordinates of the previous positions of robots
    
#### Return:
* **sense&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;bool**
    * Order sense of vertices 

In [ ]:
def getSense(vertices, previous_position):
    #print("Vertex 0 type:", type(vertices[0]))
    #print("Vertex 1 type:", type(vertices[1]))
    #print("Vertex 2 type:", type(vertices[2]))
    #print("Robots path type:", type(previous_position),"\n")
    # Check the type of cell
    if len(vertices) > 2:
        midpoint = vertices[1]
        extreme1 = vertices[0] - midpoint
        extreme2 = vertices[2] - midpoint
    else:
        midpoint = previous_position
        extreme1 = vertices[0] - midpoint
        extreme2 = vertices[1] - midpoint
    # Analyze the crossproduct
    crossproduct = extreme1[0]*extreme2[1] - extreme2[0]*extreme1[1]
    if crossproduct > 0:
        sense = 'clockwise'
    elif crossproduct < 0:
        sense = 'counterclockwise'
    return sense

### Point in a Segment Function

`def isBetween(a, b, c)`

Check inclusion of a point in a segment

#### Parameters:
* **a&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;list of float**
    * First vertex of the segment.
* **b&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;list of float**
    * Second vertex of the segment.
* **c&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;list of float**
    * Point to check.    
#### Return:
*  **True/False depending on the inclusion of the point inside the line**

In [ ]:
def isBetween(a, b, c):
    crossproduct = (c[1] - a[1])*(b[0] - a[0]) - (c[0] - a[0])*(b[1] - a[1])
    # compare versus epsilon for floating point values, or != 0 if using
    # integers
    if abs(crossproduct) > 0.01:
        return False
    dotproduct = (c[0] - a[0])*(b[0] - a[0]) + (c[1] - a[1])*(b[1] - a[1])
    if dotproduct < 0:
        return False
    squaredlengthba = (b[0] - a[0])*(b[0] - a[0]) + (b[1] - a[1])*(b[1] - a[1])
    if dotproduct > squaredlengthba:
        return False
    return True

### Deadlock Function

`def deadlock(pos, bvc, parameters, robots_path, deadlock_pointers, verbose)`

Check if there are robots in deadlock situation

#### Parameters:
* **pos&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;dictionary**
    * Robots positions and distances
* **bvc&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;dictionary**
    * Information of BVC
* **parameters&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;dictionary**
    * Global parameters
* **robots_path&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;list of ndarryas of list of floats**
    * Previous positions
* **deadlock_pointers&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of ints**
    * Pointers to robots in deadlock
* **verbose&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;boolean**
    * Flag for verbose
    
#### Return:
* **fixed_up_positions&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of floats**
    * Positions to avoid deadlock

In [ ]:
def deadlock(pos, bvc, parameters, robots_path, deadlock_pointers, verbose):
    # List to return
    fixed_up_positions = []
    # Check all robots positions
    for pointer in range(len(pos)):
        # Check if the robot is in deadlock situation
        if pointer in deadlock_pointers:
            # Initialize the instantaneous set of BVC vertices
            inst_vertices = bvc["vertices"][pointer]
            # Get the sense of the vertices_bvc array
            sense = getSense(inst_vertices, robots_path[-2][pointer])
            #i = 0
            milestone = -1
            #--------------------------CONTROL POINT--------------------------
            if verbose:
                print('Robot in deadlock:', pos[pointer])
                print('Vertices to check:', inst_vertices)
                print('Sense:', sense)
            main_distance = np.linalg.norm(bvc["closer"][pointer]
                                           - inst_vertices[0])
            for i in range(len(inst_vertices)):
                temp_distance = np.linalg.norm(bvc["closer"][pointer]
                                               - inst_vertices[i])
                if verbose:
                    print('Distance from vertex to the closest point:',
                          temp_distance)
                if (temp_distance < parameters["tolerance"]
                    and temp_distance < main_distance):
                    milestone = i
                    main_distance = temp_distance
                j = 0
                for j in range(parameters["previous"]):
                    prev_dist = np.linalg.norm(inst_vertices[i]
                                               - robots_path[-2-j][pointer])
                    if verbose:
                        print('Distance from the ' + str(j)
                              + 'th previus point:', prev_dist)
                    if (prev_dist < parameters["tolerance"]
                        and prev_dist < main_distance):
                        milestone = i
                        main_distance = prev_dist
            #--------------------------CONTROL POINT--------------------------
            if verbose:
                print('Pointer:',milestone)
            if milestone > -1 and milestone < len(inst_vertices):
                #------------------------CONTROL POINT------------------------
                if verbose:
                    print('Deadlock in vertex')
                if sense == 'counterclockwise':
                    if milestone == 0:
                        aux_vertex = (inst_vertices[-2]
                                      - inst_vertices[milestone])
                    else:
                        aux_vertex = (inst_vertices[milestone-1]
                                      - inst_vertices[milestone])
                    aux_vertex = aux_vertex / np.linalg.norm(aux_vertex)
                    position_candidate =(aux_vertex*parameters["deadlock_mov"]
                                         + pos[pointer])
                elif sense == 'clockwise':
                    aux_vertex = (inst_vertices[milestone+1]
                                  - inst_vertices[milestone])
                    aux_vertex = aux_vertex / np.linalg.norm(aux_vertex)
                    position_candidate = (aux_vertex*parameters["deadlock_mov"]
                                          + pos[pointer])
            # If deadlock is on an edge
            else:
                #------------------------CONTROL POINT-------------------------
                if verbose:
                    print('Deadlock in edge')
                if sense == 'clockwise':
                    aux_vertex = bvc["closer_edges"][2*pointer] - pos[pointer]
                    aux_vertex = aux_vertex / np.linalg.norm(aux_vertex)
                    position_candidate = (aux_vertex*parameters["deadlock_mov"]
                                          + pos[pointer])
                elif sense == 'counterclockwise':
                    aux_vertex = bvc["closer_edges"][2*pointer+1]-pos[pointer]
                    aux_vertex = aux_vertex / np.linalg.norm(aux_vertex)
                    position_candidate = (aux_vertex*parameters["deadlock_mov"]
                                          + pos[pointer])
            # Make sure the point is inside BVC
            if path.Path(inst_vertices).contains_point(position_candidate):
                fixed_up_positions.append(position_candidate)
            else:
                fixed_up_positions.append(pos[pointer])
            #--------------------------CONTROL POINT---------------------------
            if verbose:
                print('-----------------------------------')
        else:
            fixed_up_positions.append(bvc["closer"][pointer])
    #------------------------------CONTROL POINT------------------------------
    return np.array(fixed_up_positions)

### Deadlock Test Function

`def deadlock_test(parameters, options, bvc, pos, counter, robots_path)`

Check if there are robots in deadlock situation

#### Parameters:
* **parameters&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;dictionary**
    * Global parameters
* **options&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;dictionary**
    * General options
* **bvc&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;dictionary**
    * Information of BVC
* **pos&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;dictionary**
    * Robots positions and distances
* **counter&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;int**
    * Step
* **robots_path&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;list of ndarryas of list of floats**
    * Previous positions
    
#### Return:
* **closer_points&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of floats**
    * Positions to avoid deadlock

In [ ]:
def deadlock_test(parameters, options, bvc, pos, counter, robots_path):
    deadlock_flag = False
    deadlock_pointers = []
    if counter >= parameters["previous"]:
        for i in range(parameters["robots"]):
            if not np.all(bvc["closer"][i] == pos["fin"][i]):
                j = 0
                deadlock_local_flag = False
                distance1 = np.linalg.norm(bvc["closer"][i]
                                           - pos["init"][i])
                if options["verbose"]:
                    print('Distance from', options["palette"][i],
                          'to the closest point:', distance1)
                while (not deadlock_local_flag and j < parameters["previous"]):
                    distance2 = np.linalg.norm(bvc["closer"][i]
                                               - robots_path[-2-j][i])
                    if options["verbose"]:
                        print('Distance from the '+str(j+1)+
                              'th previous point:', distance2)
                    if (distance1 < parameters["tolerance"]
                        or distance2 < parameters["tolerance"]):
                        deadlock_pointers.append(i)
                        deadlock_local_flag = True
                        deadlock_flag = True
                    j += 1
    if options["verbose"]:
        print('Closer positions:', bvc["closer"])
        print('Deadlock?:', deadlock_flag,'\n')
    # BREAKING DEADLOCK
    if deadlock_flag:
        if options["verbose"]:
            print('Cells in Deadlock:')
            for i in deadlock_pointers:
                print(options["palette"][i], end = ' ')
            print('\nCloser_edges:', bvc["closer_edges"], '\n')
        bvc["closer"] = deadlock(pos["init"], bvc, parameters,
                                        robots_path, deadlock_pointers,
                                        options["verbose"])
        if options["verbose"]:
            print('DEADLOCK ABOVE!')
            print('Previous positions:', robots_path[-2])
            print('New closer positions:', bvc["closer"],'\n')
    return bvc["closer"] , deadlock_pointers

### Intersection Function

`def intersect(vertice1, origin, vertice2, radius):`

Generate the new BVC vertices from the intersection of the Voronoi edges  
moved according the safety radius in different sectors of the cell

#### Parameters:
* **vertice1&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of float**
    * Coordinates from one of the sector extreme Voronoi vertex.
* **origin&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of float**
    * Coordinates from one of the sector intermediate Voronoi vertex.
* **vertice2&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of float**
    * Coordinates from one of the sector extreme Voronoi vertex.
* **radius&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;float**
    * radius to define the BVC from the Voronoi Diagram

#### Return:
* **new_vertice1&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of float**
    * Coordinates from one of the sector extreme BVC vertex.
* **intersection&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of float**
    * Coordinates from one of the sector intermediate BVC vertex.
* **new_vertice2&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of float**
    * Coordinates from one of the sector extreme BVC vertex.

In [4]:
def intersect(vertice1, origin, vertice2, radius):
    # Change coordinates
    vertice1 = vertice1 - origin
    vertice2 = vertice2 - origin
    # Unit vectors
    vertice1_unit = vertice1 / np.linalg.norm(vertice1)
    vertice2_unit = vertice2 / np.linalg.norm(vertice2)
    # Get the angle between edges
    p_dot = np.dot(vertice1_unit, vertice2_unit)
    angle = np.arccos(p_dot)
    mid_angle = angle/2
    # Compute and show the intersection of the BVC
    mid_vertex = (vertice1_unit + vertice2_unit) / 2
    mid_vertex_unit = mid_vertex / np.linalg.norm(mid_vertex)
    separation = radius / np.sin(mid_angle)
    intersection = mid_vertex_unit * separation
    # Revert coordinates
    new_vertice1 = vertice1 + intersection + origin
    new_vertice2 = vertice2 + intersection + origin
    intersection = intersection + origin
    return [new_vertice1, intersection, new_vertice2]

### Closest Point Function

`def search_closest_point(target_point, vertices)`

Search the closest point for a robot in its BV cell. This closest point  
is either a vertex or a edge if the direction from the robot to the  
target position is perpendicular to this edge.

#### Parameters:
* **target_point&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of float**
    * The target point of a specific robot (It should be a global variable).
* **vertices&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of float**
    * Finite BVC vertices.
    
#### Return:
* **vertices\[i\]&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of float**
    * Closest vertex of the BVC to the target

In [64]:
def search_closest_point(target_point, vertices):
    # Initialize
    closest_point = np.linalg.norm(vertices[0] - target_point)
    pointer = 0
    # Review the rest of vertices
    for i in range(1,len(vertices)):
        # Measure the distance of the candidate vertex to the target
        candidate = np.linalg.norm(vertices[i] - target_point)
        # Take the closest point
        if candidate < closest_point:
            closest_point = candidate
            pointer = i
    # Return the closest vertex 
    return vertices[pointer]

### Closer Edge Points Function

`def perpendicular_distance(target_point, vertices)`

Get the closest point of an edge to the target point

#### Parameters:
* **target_point&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of float**
    * The target point of a specific robot.
* **vertices&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of float**
    * Finite BVC vertices.
    
#### Return:
* **edge_points&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of float**
    * List of closest points on the BVC edges to the target

In [ ]:
def perpendicular_distance(target_point, vertices):
    #Initialize the list to return
    edge_points = []
    # Analize each edge
    for i in range(len(vertices[:-1])):
        # Check if the edge is vertical
        if vertices[i][0] == vertices[i+1][0]:
            [x, y] = [vertices[i][0], target_point[1]]
        else:
            # Get the perpendicular point on the egde sense to the target
            slope = (vertices[i+1][1] - vertices[i][1])/(vertices[i+1][0]
                                                         - vertices[i][0])
            x = (slope * (vertices[i][1] - target_point[1]
                          - slope * vertices[i][0])
                 - target_point[0]) / (-1 - pow(slope,2))
            y = (x - target_point[0])/(-slope) + target_point[1]
            # Check if the point is inside of edge to include in the returning
            # list
        if isBetween(vertices[i], vertices[i+1], [x,y]):
            edge_points.append([x,y])
    return edge_points

### Critical Distance Function

`def approved_distance(node, radius, vertices, sense)`

Check if the safety radius length does not create conflicts with the edge lenght

#### Parameters:
* **node&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;node_linker.Node**
    * The start node of the analysis that points to a Voronoi vertex in a determined cell.
* **radius&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;float**
    * radius to define the BVC from the Voronoi Diagram
* **vertices&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of float**
    * Updated vertices of Voronoi Diagram with approximation of infinity vertices
* **sense&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;string**
    * Back or forward sense

#### Return:
* **approve&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;boolean**
    * True if there is not a problem

In [ ]:
def approved_distance(node, radius, vertices, sense, verbose):
    coordinate2 = vertices[node.item]
    # Primarly for open cells
    if sense == 'forward':
        coordinate1 = vertices[node.pref.item]
        coordinate3 = vertices[node.nref.item]
        coordinate4 = vertices[node.nref.nref.item]
    # For closed cells
    elif sense == 'back':
        coordinate1 = vertices[node.nref.item]
        coordinate3 = vertices[node.pref.item]
        coordinate4 = vertices[node.pref.pref.item]  
    # Use the Intersections Formula and Sharpen Function in order to reach the
    # critical distance
    inst1 = intersect(coordinate1, coordinate2, coordinate3, radius)
    inst2 = intersect(coordinate2, coordinate3, coordinate4, radius)
    critical_point = sharpen(coordinate2, inst1[1], coordinate3, inst2[1])
    approve = not isBetween(coordinate2, inst1[1], critical_point)
    if verbose:
        print('\nChecking distances in',sense,'sense')
        print('Critical point:',critical_point)
        print('First segment: {',coordinate2,',',inst1[1],'}')
        print('Second segment: {',coordinate3,',',inst2[1],'}')
        print('Approve?',approve,'\n')
    return approve

### Pruner Function

`def pruner(vertices, verbose)`

Remove intersections of the BVC

#### Parameters:
* **vertices&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of float**
    * BVC vertices
* **verbose&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;boolean**
    * Flag to print important values

#### Return:
* **pruned_vertices&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of float**
    * BVC vertices without intersections

In [ ]:
def pruner(vertices, verbose, mode):
    pruned_vertices = []
    cell = node_linker.DoublyLinkedList()
    for vertex in vertices:
        cell.insert_at_end(vertex)
    runner = cell.start_node
    if verbose:
        print('Vertices in doubly linked list:')
        cell.traverse_list()
    while not(runner is cell.end_node.pref.pref
          or runner is cell.end_node.pref):
        vanguard = runner.nref.nref
        while vanguard is not cell.end_node:
            vertex_0 = runner.item
            vertex_1 = runner.nref.item
            vertex_2 = vanguard.item
            vertex_3 = vanguard.nref.item
            if verbose:
                print("\nChecking: ",vertex_0,"—",vertex_1,"\nand ",
                      vertex_2,"—",vertex_3)
            critical_point = sharpen(vertex_0,vertex_1,vertex_2,vertex_3)
            intersection = isBetween(vertex_0,vertex_1,critical_point)
            if verbose:
                print('Intersection?:',intersection)
            if intersection:
                inst_node = runner.nref
                while inst_node is not vanguard:
                    inst_node = inst_node.nref
                    cell.delete_node(inst_node.pref)
                vanguard.item = critical_point
                if verbose:
                    print('The new value is: ',vanguard.item)
                    print('Connected with: ',vanguard.nref.item)
            vanguard = vanguard.nref
        runner = runner.nref
    runner = cell.start_node
    while runner is not cell.end_node.nref:
        pruned_vertices.append(runner.item)
        runner = runner.nref
    if verbose:
        print("\nCell size: ", cell.lenght_list)
        cell.traverse_list()
    return pruned_vertices

### Voronoi Diagram Generator Method

`def voronoi_approx(robots, vor, verbose)`

Generate the main Voronoi Diagram approximating the values of infinity vertices  
and returning the updated values of edges and vertices for BVC constructor.

#### Parameters:
* **robots&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of floats**
    * Robots position
* **vor&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;class scipy.spatial.Voronoi**
    * Voronoi diagram in N dimension
* **verbose&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;bool**
    * Flag for verbose
    
#### Return:
* **vertices&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of float**
    * Updated vertices with approximation of infinity vertices
* **edges&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of int**
    * Updated pointers to vertices for each edge

In [40]:
def voronoi_approx(robots, vor, verbose):
    counter = 0
    # Create copies of:
    vertices = vor.vertices
    edges = []
    # Find the center of the robots position
    center = robots.mean(axis=0)
    # pointidx: Indices of the points between which each Voronoi ridge lies.
    # simplex: Indices of the Voronoi vertices forming each Voronoi ridge.
    # zip join each values in a tuple
    for pointidx, simplex in zip(vor.ridge_points, vor.ridge_vertices):
        # Transform to array
        simplex_arr = np.array(simplex)
        # If the Voronoi edge goes to infinity
        if np.all(simplex_arr >= 0):
            # Append to the BVC edges list
            edges.append(simplex)
        # If one of the vertices goes to the infinity
        else:
            # Get the finite vertex
            i = simplex_arr[simplex_arr >= 0][0] 
            # Calculate the straight line between robots
            t = robots[pointidx[1]] - robots[pointidx[0]] 
            # Normalization of this line
            t = t / np.linalg.norm(t)
            # Ortogonalization
            n = np.array([-t[1], t[0]])
            # Get the midpoint between robots
            midpoint = robots[pointidx].mean(axis=0)
            # Far point = extension from the finite vertex using the
            #             orthogonalization and
            #             depending of the midpoint location respect center
            far_point = vor.vertices[i] + (np.sign(np.dot(midpoint- center,n))
                                           * n * 20 * len(robots))
            # Increase the counter
            counter += 1
            # Add the value to the BVC vertices array
            vertices = np.concatenate((vertices, far_point.reshape(1,2)),
                                      axis=0)
            # Update the pointer of BVC edges list 
            edges.append([len(vertices)-1, simplex[1]])
    if verbose:
        print('List of vertex coordinates (including approximations) of',
              'Voronoi Diagram:\n')
        for i in range(len(vertices)):
            print(i,".-",vertices[i])
        print('Robots in front: ',vor.ridge_points,'\n')
        print('Voronoi Diagram Edges: ',edges,'\n')
    return {"vrtx":vertices, "edge":edges}

### Palette Function

`def colors(number):`

Create the color palette

#### Parameters:
* **number&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;int**
    * Number of colors to retun
    
#### Return:
* **palette&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;list of str**
    * A list with the colors name

In [1]:
def colors(number):
    overlap = ['red','tomato','orangered','orange','goldenrod','gold',
               'yellowgreen','chartreuse','lime','green','darkgreen','teal',
               'turquoise','aqua','cyan','blue','navy','indigo','purple',
               'fuchsia','magenta','crimson','maroon','brown','sienna']
    palette = []
    for i in range(number):
        palette.append(overlap[int(i*(len(overlap)/number))])
    return palette

### BVC Generator Function

`def bvc_gen(robots, target, radius, vertices, edges, bvc, verbose)`

Assemble the vertices for each cell in order to generate the new BVC vertices

#### Parameters:
* **pos&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;dictionary**
    * Robots positions and distances
* **radius&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;float**
    * radius to define the BVC from the Voronoi Diagram
* **vor_approx&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;dictionary**
    * Approximated values for Voronoi generation
* **vrtx_assembly&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;list of node_linker.DoublyLinkedList**
    * A list of ordered vertices for each robot
* **robots_path&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;list of ndarray**
    * Previous positions
* **options&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;dictionary**
    * General options
    
#### Return:
* **new_robots&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of floats**
    * New position of robots

In [2]:
def bvc_gen(pos, radius, vor_approx, vrtx_assembly, robots_path, options):
    counter = 0
    palette = colors(len(pos["init"]))
    new_robots = [] # For storing the next position of robots
    set_of_new_vertices = [] # For storing the BVC vertices of each robot
    closer_edges = [] # For save the closer edge
    # For internal color
    if options["plot"]:
        fig, ax = plt.subplots()
    if options["verbose"]:
        print("==================================================")
        print("                 BVC GENERATION")
        print("==================================================")
    # Calculate the bvc vertices and the closest point
    for index, cell in enumerate(vrtx_assembly):
        if options["verbose"]:
            print('Analyzing these vertices...')
            cell.traverse_list()
        new_vertices = []
        milestone_node = cell.start_node
        # Go in pairs of edges
        # Check if the cell is unlimited
        if cell.end_node.nref is None:
            if options["verbose"]:
                print('\nOPEN CELL!')
                print('Number of vertices in this cell:',
                      cell.lenght_list,'\n')
            while milestone_node.nref.nref is not cell.end_node.nref:
                # Auxiliar node for possible cuts
                aux_node = milestone_node.nref
                # Get coordinates
                vertex_start = vor_approx["vrtx"][milestone_node.item]
                vertex_mid1 = vor_approx["vrtx"][milestone_node.nref.item]
                vertex_mid2 = vor_approx["vrtx"][aux_node.item]
                vertex_end = vor_approx["vrtx"][aux_node.nref.item]
                # Get the new vertices in this sector
                sector = intersect(vertex_start, vertex_mid1, vertex_end,
                                   radius)
                # Assign the new values according special cases 
                if milestone_node is cell.start_node:
                    new_vertices.append(sector[0])
                new_vertices.append(sector[1])
                if aux_node.nref.nref is None:
                    new_vertices.append(sector[2])
                # Move the milestone_node 
                milestone_node = aux_node
            # Remove intersections of the BVC
            new_vertices = pruner(new_vertices,options["verbose"], "open")
        # If the cell is limited or closed
        else:
            if options["verbose"]:
                print('\nCLOSED CELL!')
                print('Number of vertices in this cell:',cell.lenght_list)
            for _ in range(cell.lenght_list):
                # Auxiliar node for possible cuts
                aux_node = milestone_node.nref
                # Get coordinates
                vertex_start = vor_approx["vrtx"][milestone_node.item]
                vertex_mid1 = vor_approx["vrtx"][milestone_node.nref.item]
                vertex_mid2 = vor_approx["vrtx"][aux_node.item]
                vertex_end = vor_approx["vrtx"][aux_node.nref.item]
                # Get the new vertices in this sector
                sector = intersect(vertex_start, vertex_mid1, vertex_end,
                                   radius)
                if options["verbose"]:
                    print(sector[1])
                new_vertices.append(sector[1])
                # Move the milestone_node 
                milestone_node = aux_node
            # Remove intersections of the BVC
            ls = LineString(new_vertices)
            lr = LineString(ls.coords[:] + ls.coords[0:1])
            if not lr.is_simple:
                if options["verbose"]:
                    print("\nThere is an intersection!")
                mls = unary_union(lr)
                for polygon in polygonize(mls):
                    if polygon.contains(Point(pos["init"][index])):
                        break;
                new_vertices = [np.array(k) for k in polygon.exterior.coords]
            else:
                new_vertices.append(new_vertices[0])
        # Check if target point is in/out of BVC
        bbPath = path.Path(np.array(new_vertices))
        inside = bbPath.contains_point(pos["fin"][counter])
        if inside:
            closest_point = pos["fin"][counter]
            closer_edges.extend([[0,0],[0,0]])
        else:
            # Calculate the closest point using the CVXPY Solver
            if options["optimize"]:
                closest_point = solver.optimizer(pos["init"][counter],
                                                 pos["fin"][counter],
                                                 np.delete(pos["init"],
                                                           counter,0),
                                                 radius, False,
                                                 options["verbose"])
                get_intersection = False
                i = 0
                while not get_intersection and i < len(new_vertices)-1:
                    if isBetween(new_vertices[i+1], new_vertices[i],
                                 closest_point):
                        get_intersection = True
                        closer_edges.extend([new_vertices[i+1],
                                             new_vertices[i]])
                    else:
                        i += 1
                if not get_intersection:
                    closer_edges.extend([[0,0],[0,0]])
            # Calculate the closest point using the Analytical Geometric
            # Algorithm
            else:
                edge_points = perpendicular_distance(pos["fin"][counter],
                                                     new_vertices)
                closest_point = search_closest_point(np.array(pos["fin"][counter]),
                                                     np.array(new_vertices
                                                              +edge_points))
                #if options["verbose"]:
                    #print('Closest point candidates:',new_vertices+edge_points)
                    #print('Lenght:',len(new_vertices+edge_points))
                    #print('THE CLOSEST POINT:',closest_point)
                get_intersection = False
                i = 0
                while not get_intersection and i < len(new_vertices)-1:
                    closest_point = sharpen(pos["fin"][counter], pos["init"][counter],
                                            new_vertices[i+1],
                                            new_vertices[i])
                    is_in_edge = isBetween(new_vertices[i+1], new_vertices[i],
                                           closest_point)
                    is_in_path = isBetween(pos["fin"][counter], pos["init"][counter],
                                           closest_point)
                    if is_in_edge and is_in_path:
                        get_intersection = True
                        closer_edges.extend([new_vertices[i+1],
                                             new_vertices[i]])
                    else:
                        i += 1
                if not get_intersection:
                    closer_edges.extend([[0,0],[0,0]])
            if options["verbose"]:
                print('Closest point:',closest_point)
        # Store the closest point
        new_robots.append(closest_point)
        # Save the new vertices
        set_of_new_vertices.append(new_vertices)
        if options["verbose"]:
            print('The new vertices are:',type(new_vertices[0]))
            for i in range(len(new_vertices)):
                print(new_vertices[i])
            print('---------------------------------------------------------')
        if options["plot"]:
            # Set the color
            cell_color = palette[counter]
            patches = []
            #plt.xlim((math.sqrt(2*len(vrtx_assembly))
            #          +(1/(0.4*len(vrtx_assembly))))*-1.4,
            #         (math.sqrt(2*len(vrtx_assembly))
            #           +(1/(0.4*len(vrtx_assembly))))*1.4)
            #plt.ylim(-(math.sqrt(2*len(vrtx_assembly))
            #           +(1/(0.4*len(vrtx_assembly)))),
            #          (math.sqrt(2*len(vrtx_assembly))
            #           +(1/(0.4*len(vrtx_assembly)))))
            plt.xlim(-1.05,1.05); plt.ylim(-0.75,0.75)
            #plt.xlim(-0.53,0.53); plt.ylim(-0.38,0.38)
            # Display edges
            for i in range(len(new_vertices)-1):
                plt.plot([new_vertices[i][0], new_vertices[i+1][0]],
                         [new_vertices[i][1], new_vertices[i+1][1]],
                         color = cell_color, linestyle = '-')
            # Plot robots, target, line to target,robot path and new direction
            plt.plot(pos["init"][counter,0], pos["init"][counter,1],
                     color = cell_color, marker = 'o')
            plt.plot(pos["fin"][counter,0], pos["fin"][counter,1],
                     color = cell_color,
                     marker = 'o')
            plt.plot([pos["init"][counter,0], pos["fin"][counter,0]],
                     [pos["init"][counter,1], pos["fin"][counter,1]],
                     color = 'gray', linestyle = '--')
            # BVC Cell inside
            inst_polygon = pltpolygon(np.array(new_vertices), True)
            patches.append(inst_polygon)
            p = PatchCollection(patches, alpha=0.1)
            p.set_color(cell_color)
            ax.add_collection(p)
            # Plot path
            for i in range(len(robots_path)-1):
                inst_start = robots_path[i][counter]
                inst_end = robots_path[i+1][counter]
                plt.plot([inst_start[0], inst_end[0]], [inst_start[1],
                                                        inst_end[1]],
                         color = cell_color, linestyle = '-')
            #plt.plot()
            plt.plot([pos["init"][counter,0], closest_point[0]],
                     [pos["init"][counter,1], closest_point[1]],
                     'k-')
            # Display the Voronoi Diagram
            for arista in vor_approx["edge"]:
                x_values = [vor_approx["vrtx"][arista[0],0],
                            vor_approx["vrtx"][arista[1],0]]
                y_values = [vor_approx["vrtx"][arista[0],1],
                            vor_approx["vrtx"][arista[1],1]]
                plt.plot(x_values, y_values, color = 'gray', linestyle = '-')
        # Increase the counter
        counter += 1
    return {"closer": np.array(new_robots),
            "vertices"     : np.array(set_of_new_vertices),
            "closer_edges" : np.array(closer_edges)
           }